In [1]:
from pyspark.sql import SparkSession, functions as f
import os

In [2]:
if not os.path.exists("../../../data/insights/agg_insight_data"):
    os.makedirs("../../../data/insights/agg_insight_data")

In [3]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/30 17:23:46 WARN Utils: Your hostname, DulanComputer resolves to a loopback address: 127.0.1.1; using 172.30.15.25 instead (on interface eth0)
23/09/30 17:23:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/30 17:23:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
merchant = spark.read.parquet("../../../data/insights/pre_insights/merchant.parquet/")

In [5]:
merchant.show(truncate= False)

+------------+-------------------------------------+----------------------+--------------------+----------------------------+
|merchant_abn|merchant_name                        |merchant_revenue_level|merchant_take_rate_%|merchant_fraud_probability_%|
+------------+-------------------------------------+----------------------+--------------------+----------------------------+
|16570599421 |Non Magna Nam PC                     |a                     |6.74                |0.0                         |
|63290521567 |Vehicula Pellentesque Corporation    |a                     |6.48                |0.0                         |
|13514558491 |Magna Praesent PC                    |a                     |6.78                |0.0                         |
|22227727512 |Malesuada Integer Id Foundation      |b                     |5.03                |0.0                         |
|79827781481 |Amet Risus Inc.                      |a                     |6.82                |0.0                   

In [6]:
merchant.dtypes

[('merchant_abn', 'bigint'),
 ('merchant_name', 'string'),
 ('merchant_revenue_level', 'string'),
 ('merchant_take_rate_%', 'float'),
 ('merchant_fraud_probability_%', 'double')]

In [7]:
merchant.select(merchant.merchant_name, merchant.merchant_revenue_level).distinct().count()

4026

In [8]:
merchant_revenue_level_count = merchant.groupBy("merchant_abn").agg(f.countDistinct("merchant_revenue_level").alias("revenue_level_count"))
merchant_take_rate_count = merchant.groupBy("merchant_abn").agg(f.countDistinct("merchant_take_rate_%").alias("take_rate_count"))

In [9]:
print(merchant_revenue_level_count.filter(merchant_revenue_level_count.revenue_level_count > 1).count())
print(merchant_take_rate_count.filter(merchant_take_rate_count.take_rate_count > 1).count())

0


0


In [10]:
merchant_aggregated = merchant.groupBy("merchant_abn").agg(\
                        f.first("merchant_name").alias("name"),
                        f.first("merchant_revenue_level").alias("revenue_level"),
                        f.first("merchant_take_rate_%").alias("take_rate"),
                        f.avg("merchant_fraud_probability_%").alias("average_merchant_fraud_probability"))

In [11]:
merchant_aggregated.show()

+------------+--------------------+-------------+---------+----------------------------------+
|merchant_abn|                name|revenue_level|take_rate|average_merchant_fraud_probability|
+------------+--------------------+-------------+---------+----------------------------------+
| 10023283211|       Felis Limited|            e|     0.18|                               0.0|
| 10142254217|Arcu Ac Orci Corp...|            b|     4.22|                               0.0|
| 10165489824|    Nunc Sed Company|            b|      4.4|                               0.0|
| 10187291046|Ultricies Digniss...|            b|     3.29|                               0.0|
| 10192359162| Enim Condimentum PC|            a|     6.33|                               0.0|
| 10206519221|       Fusce Company|            a|     6.34|                               0.0|
| 10255988167|Aliquam Enim Inco...|            b|     4.32|                               0.0|
| 10264435225|    Ipsum Primis Ltd|            c| 

In [12]:
merchant_aggregated.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/merchant_agg.parquet")

In [13]:
spark.stop()